In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# __file__ = '/content/drive/MyDrive/aiffelthon/data'
__file__ = '/content/drive/Othercomputers/MacBook_Air/path-finding-rl/data'

Mounted at /content/drive


In [5]:
from string import ascii_uppercase
#from draw_utils import *
#from pyglet.gl import *
import numpy as np
import pandas as pd
import os
import random
from datetime import datetime
import pytz
import matplotlib.pyplot as plt

# reward
move_reward = -0.1
obs_reward = -10
goal_reward = 100
###################################
# train or test 모드 지정
train_mode = True
###################################
print('reward:' , move_reward, obs_reward, goal_reward)

#__file__ = '/home/ogangza/heung_path_finding/path-finding-rl/data'

local_path = os.path.abspath(os.path.join(os.path.dirname(__file__)))


class Simulator:
    def __init__(self):
        '''
        height : 그리드 높이
        width : 그리드 너비 
        inds : A ~ Q alphabet list
        '''
        #######################################################################################
        # Load train or test data
        if train_mode:  # 훈련 데이타 읽기
            self.files = pd.read_csv(os.path.join(local_path, "data/factory_order_train.csv"))
            print('data/factory_order_train.csv used')
        else:  # 테스트 데이터 읽기
            self.files = pd.read_csv(os.path.join(local_path, "data/factory_order_test.csv"))
            print('data/factory_order_test.csv used')
        #######################################################################################
        self.height = 10
        self.width = 9
        self.inds = list(ascii_uppercase)[:17]

    def set_box(self):
        '''
        아이템들이 있을 위치를 미리 정해놓고 그 위치 좌표들에 아이템이 들어올 수 있으므로 그리드에 100으로 표시한다.
        데이터 파일에서 이번 에피소드 아이템 정보를 받아 가져와야 할 아이템이 있는 좌표만 -100으로 표시한다.
        self.local_target에 에이전트가 이번에 방문해야할 좌표들을 저장한다.
        따라서 가져와야하는 아이템 좌표와 end point 좌표(처음 시작했던 좌표로 돌아와야하므로)가 들어가게 된다.
        '''
        box_data = pd.read_csv(os.path.join(local_path, "./data/box.csv"))
        
        ######################################
        #print('box data:', box_data)
        ######################################

        # 물건이 들어있을 수 있는 경우
        for box in box_data.itertuples(index = True, name ='Pandas'):
            self.grid[getattr(box, "row")][getattr(box, "col")] = 100
            
            #######################################
            #print('self.grid:', self.grid)
            #######################################

        # 물건이 실제 들어있는 경우
        order_item = list(set(self.inds) & set(self.items))
        order_csv = box_data[box_data['item'].isin(order_item)]
        
        ########################################
        #print('order_csv:', order_csv)
        ########################################

        for order_box in order_csv.itertuples(index = True, name ='Pandas'):
            self.grid[getattr(order_box, "row")][getattr(order_box, "col")] = -100
            # local target에 가야 할 위치 좌표 넣기
            self.local_target.append(
                [getattr(order_box, "row"),
                 getattr(order_box, "col")]
                )

            #######################################
            #print('self.grid:', self.grid)
            #######################################

        ################################################
        #print('self.local_target.sort() 전:', self.local_target)
        ################################################
        
        # self.local_target.sort() 
        self.local_target.append([9,4]) 

        ################################################
        #print('self.local_target.sort():', self.local_target)
        ###############################################
        
        # 알파벳을 Grid에 넣어서 -> grid에 2Dconv 적용 가능

    def set_obstacle(self):
        '''
        장애물이 있어야하는 위치는 미리 obstacles.csv에 정의되어 있다. 이 좌표들을 0으로 표시한다.
        '''
        obstacles_data = pd.read_csv(os.path.join(local_path, "./data/obstacles.csv"))
        for obstacle in obstacles_data.itertuples(index = True, name ='Pandas'):
            self.grid[getattr(obstacle, "row")][getattr(obstacle, "col")] = 0
            
        ##########################################
        #print('self.grid:', self.grid)
        ##########################################
        
    def reset(self, epi):
        '''
        reset()은 첫 스텝에서 사용되며 그리드에서 에이전트 위치가 start point에 있게 한다.

        :param epi: episode, 에피소드 마다 가져와야 할 아이템 리스트를 불러올 때 사용
        :return: 초기셋팅 된 그리드
        :rtype: numpy.ndarray
        _____________________________________________________________________________________
        items : 이번 에피소드에서 가져와야하는 아이템들
        terminal_location : 현재 에이전트가 찾아가야하는 목적지
        local_target : 한 에피소드에서 찾아가야하는 아이템 좌표, 마지막 엔드 포인트 등의 위치좌표들
        actions: visualization을 위해 에이전트 action을 저장하는 리스트
        curloc : 현재 위치
        '''

        # initial episode parameter setting
        self.epi = epi
        self.items = list(self.files.iloc[self.epi])[0]
        self.cumulative_reward = 0
        self.terminal_location = None
        self.local_target = []
        self.actions = []

        # initial grid setting
        self.grid = np.ones((self.height, self.width), dtype="float16")

        # set information about the gridworld
        self.set_box()
        self.set_obstacle()

        # start point를 grid에 표시
        self.curloc = [9, 4]
        self.grid[int(self.curloc[0])][int(self.curloc[1])] = -5
        
        self.done = False
        
        ########################################
        #print('reset curloc:', self.curloc)
        ########################################
        
        return self.grid

    def apply_action(self, action, cur_x, cur_y):
        '''
        에이전트가 행한 action대로 현 에이전트의 위치좌표를 바꾼다.
        action은 discrete하며 4가지 up,down,left,right으로 정의된다.
        
        :param x: 에이전트의 현재 x 좌표
        :param y: 에이전트의 현재 y 좌표
        :return: action에 따라 변한 에이전트의 x 좌표, y 좌표
        :rtype: int, int
        '''
        new_x = cur_x
        new_y = cur_y
        # up
        if action == 0:
            new_x = cur_x - 1
        # down
        elif action == 1:
            new_x = cur_x + 1
        # left
        elif action == 2:
            new_y = cur_y - 1
        # right
        else:
            new_y = cur_y + 1

        return int(new_x), int(new_y)


    def get_reward(self, new_x, new_y, out_of_boundary):
        '''
        get_reward함수는 리워드를 계산하는 함수이며, 상황에 따라 에이전트가 action을 옳게 했는지 판단하는 지표가 된다.

        :param new_x: action에 따른 에이전트 새로운 위치좌표 x
        :param new_y: action에 따른 에이전트 새로운 위치좌표 y
        :param out_of_boundary: 에이전트 위치가 그리드 밖이 되지 않도록 제한
        :return: action에 따른 리워드
        :rtype: float
        '''

        # 바깥으로 나가는 경우
        if any(out_of_boundary):
            reward = obs_reward
                       
        else:
            # 장애물에 부딪히는 경우 
            if self.grid[new_x][new_y] == 0:
                reward = obs_reward  

            # 현재 목표에 도달한 경우
            elif new_x == self.terminal_location[0] and new_y == self.terminal_location[1]:
                reward = goal_reward

            # 그냥 움직이는 경우 
            else:
                reward = move_reward

        return reward

    def step(self, action):
        ''' 
        에이전트의 action에 따라 step을 진행한다.
        action에 따라 에이전트 위치를 변환하고, action에 대해 리워드를 받고, 어느 상황에 에피소드가 종료되어야 하는지 등을 판단한다.
        에이전트가 endpoint에 도착하면 gif로 에피소드에서 에이전트의 행동이 저장된다.

        :param action: 에이전트 행동
        :return:
            grid, 그리드
            reward, 리워드
            cumulative_reward, 누적 리워드
            done, 종료 여부
            goal_ob_reward, goal까지 아이템을 모두 가지고 돌아오는 finish율 계산을 위한 파라미터

        :rtype: numpy.ndarray, float, float, bool, bool/str

        (Hint : 시작 위치 (9,4)에서 up말고 다른 action은 전부 장애물이므로 action을 고정하는 것이 좋음)
        '''

        #print('self.local_target:', self.local_target)
        self.terminal_location = self.local_target[0]
        cur_x,cur_y = self.curloc
        self.actions.append((cur_x, cur_y))

        goal_ob_reward = False
        
        new_x, new_y = self.apply_action(action, cur_x, cur_y)

        out_of_boundary = [new_x < 0, new_x >= self.height, new_y < 0, new_y >= self.width]

        # 바깥으로 나가는 경우 종료
        if any(out_of_boundary):
            
            #원 위치
            #new_x = cur_x
            #new_y = cur_y
            
            self.done = True
            goal_ob_reward = True
            
        else:
            # 장애물에 부딪히는 경우 종료
            if self.grid[new_x][new_y] == 0:
                
                #원 위치
                #new_x = cur_x
                #new_y = cur_y
                
                self.done = True
                goal_ob_reward = True

            # 현재 목표에 도달한 경우, 다음 목표 설정
            elif new_x == self.terminal_location[0] and new_y == self.terminal_location[1]:

                # end point 일 때
                if [new_x, new_y] == [9,4]:
                    self.done = True

                self.local_target.remove(self.local_target[0])
                self.grid[cur_x][cur_y] = 1
                self.grid[new_x][new_y] = -5
                goal_ob_reward = True
                self.curloc = [new_x, new_y]
            else:
                # 그냥 움직이는 경우 
                self.grid[cur_x][cur_y] = 1
                self.grid[new_x][new_y] = -5
                self.curloc = [new_x,new_y]
                
        reward = self.get_reward(new_x, new_y, out_of_boundary)
        self.cumulative_reward += reward
        return self.grid, reward, self.cumulative_reward, self.done, goal_ob_reward


reward: -0.1 -10 100


In [6]:
import gym
import collections
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# path
# model_path = __file__ + '/model.pt'

#Hyperparameters
learning_rate = 0.0005
gamma         = 0.98
buffer_limit  = 100000 # 5만 -> 10만
batch_size    = 32

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)
    
    def put(self, transition):
        self.buffer.append(transition)
    
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return torch.tensor(np.array(s_lst), dtype=torch.float), torch.tensor(np.array(a_lst)), \
               torch.tensor(np.array(r_lst)), torch.tensor(np.array(s_prime_lst), dtype=torch.float), \
               torch.tensor(np.array(done_mask_lst))
    
    def size(self):
        return len(self.buffer)

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(94, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 4)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
      
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0,3)
        else : 
            return out.argmax().item()

    ###################################################
    def test_action(self, obs):  # Test용 action 추가
        out = self.forward(obs)
        return out.argmax().item()
    ###################################################
            
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)

        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def main():
    tz = pytz.timezone('Asia/Seoul')

    env = Simulator()
    q = Qnet()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()

    print_interval = 1000
    score = 0.0  
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)
    

    #files = pd.read_csv("./data/factory_order_train.csv") # Simulator 클래스 __init__(self):에서 읽음
    print('len(env.files):', len(env.files))
    
    #path_length = []

    for n_epi in range(len(env.files)): # range()의 인수로 len(env.files) 사용하면 됨 (=39,999)
        epsilon = max(0.01, 0.99 - 0.01*(n_epi/200)) # 상민님 epi/40000
        env.grid = env.reset(n_epi)  # env reset: 에피소드 번호에 해당하는 목표물 리스트 env.local_target 생성, 그리드맵 생성
        
        start_point = [9,4]  # 출발지 지정
        end_point = env.local_target[0]  # 첫 번째 목적지 지정
        #print('list(env.files.iloc[',n_epi,'])[0]:', list(env.files.iloc[n_epi])[0])
        ##print('env.local_target:', env.local_target)

        #######################################################################
        ## 인풋 정보: 그리드맵, 출발지, 목적지
        #######################################################################
        # 그리드 맵 지정
        grid_map = env.grid.reshape(-1)          # 그리드 맵
        # 출발지, 목적지 좌표 지정
        #env.x, env.y = start_point                # 출발지 좌표
        #env.end_x, env.end_y = end_point          # 목적지 좌표
        #######################################################################
        # 목적지 도착 flag (done) 및 인풋 데이터 (s) 초기화
        done = False
        s_temp_1 = np.append(grid_map, start_point)
        s_temp_2 = np.append(s_temp_1, end_point)
        s = np.array(s_temp_2)
        old_score = 0.0

        while not done:
            a = q.sample_action(torch.from_numpy(s).float(), epsilon)
            grid, r, cum_reward, done, goal_ob_reward = env.step(a)
            #s_prime = np.array(env.curloc)
            new_grid_map = grid.reshape(-1)
            s_prime_temp_1 = np.append(new_grid_map, env.curloc)
            s_prime_temp_2 = np.append(s_prime_temp_1, end_point)
            s_prime = np.array(s_prime_temp_2)

            done_mask = 0.0 if done else 1.0
            memory.put((s,a,r/10.0,s_prime, done_mask))
            s = s_prime

            score += r
            '''
            if score > old_score:
                    cur_time = datetime.now(tz)
                    simple_cur_time = cur_time.strftime("%H:%M:%S")
                    print('▶ Episode #', n_epi, 'start time:', simple_cur_time, end='→')
                    print('Score =', score, 'appended actions =', s, end='...')
                    print('경로 길이:', len(env.actions))
            '''
            old_score = score
                       
            if done:
                break

            # while loop 종료

        ############################################################################
                ## memory.size(): 2000 → 10000 → 20000 수정... 이후 훈련 시작
        ############################################################################   
        if memory.size() > 10000: # 20000 -> 10000
            train(q, q_target, memory, optimizer)

        if n_epi % print_interval == 0 and n_epi != 0:
            cur_time = datetime.now(tz)
            simple_cur_time = cur_time.strftime("%H:%M:%S")
            print('▶ Episode #', n_epi, 'start time:', simple_cur_time, end='→')
            q_target.load_state_dict(q.state_dict())
            print("n_episode :{}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(
                                                            n_epi, score/print_interval, memory.size(), epsilon*100))
            print('경로 길이:', len(env.actions))
            
            score = 0.0

            # torch.save(q, model_path)
            torch.save(q, __file__+f"/{n_epi}_model.pt")
        ##### 액션 결과물 확인하기
        if len(env.actions) > 10:
            print(f"move = {move_reward}, obs = {obs_reward}, goal = {goal_reward}")
            print(f"{n_epi}번째 에피소드는 {env.cumulative_reward}점 입니다.")
            print(f"target은 {env.items}, {env.local_target}입니다.")
            print(f"action은 총 {len(env.actions)}번 움직였습니다.\n{env.actions}")
            print("###############")
    # torch.save(q, model_path)
    torch.save(q, __file__+f"/{n_epi}_model.pt")

##########################
## 나중에 수정할 것!!!
##########################

# if __name__ == '__main__':
#     main()

In [9]:
def test():
    # model = torch.load('/content/drive/MyDrive/aiffelthon/data/model.pt')
    # model = torch.load(model_path)
    model_name = "/10000_model.pt"
    model = torch.load(__file__ + model_name)
    print(f"model path = {model_name}")
    train_mode = False  # False
    tz = pytz.timezone('Asia/Seoul')
    env = Simulator()
    q = Qnet()
    s_before = None
    print_interval = 1
    score = 0.0

    #################################
    # 액션 저장하는 txt파일 만들기
    # my_file_name = f"{model_name}_{move_reward}_{obs_reward}_{goal_reward}"
    # f = open(my_file_name+'.txt', 'w')
    #################################

    for n_epi in range(len(env.files)): # range()의 인수로 len(env.files) 사용하면 됨 (=1225)
        env.grid = env.reset(n_epi)
        grid_map = env.grid.reshape(-1)          # 그리드 맵
        # 출발지, 목적지 좌표 지정
        #env.x, env.y = start_point                # 출발지 좌표
        #env.end_x, env.end_y = end_point          # 목적지 좌표
        start_point = [9,4]  # 출발지 지정
        end_point = env.local_target[0]  # 첫 번째 목적지 지정
        #######################################################################
        # 목적지 도착 flag (done) 및 인풋 데이터 (s) 초기화
        done = False
        s_temp_1 = np.append(grid_map, start_point)
        s_temp_2 = np.append(s_temp_1, end_point)
        s = np.array(s_temp_2)

        while not done:  # OB, 장애물, 최종 목표 도달시 종료
            ####################################################################################
            # # -> 무한 루프
            # # (option) test 경우 추가: 출발점에서는 무조건 위로 올라간다 (action=0)
            # x, y = s[-4:-2]
            # if [x,y] == [9,4]:
            #     a = 0
            # else:
            #     a = q.test_action(torch.from_numpy(s).float())  # test_action 수행 ->
            #####################################################################################
            a = q.test_action(torch.from_numpy(s).float())  # test_action 수행
            #####################################################################################
            grid, r, cum_reward, done, goal_ob_reward = env.step(a)
            # s_prime = np.array(env.curloc)
            new_grid_map = grid.reshape(-1)
            s_prime_temp_1 = np.append(new_grid_map, env.curloc)
            s_prime_temp_2 = np.append(s_prime_temp_1, end_point)
            s_prime = np.array(s_prime_temp_2)

            s = s_prime
            score += r

        # print_interval (=1) 마다 현황 디스플레이하고 q_target 업데이트 및 score 초기화
        if n_epi % print_interval == 0 and n_epi != 0:
            cur_time = datetime.now(tz)
            simple_cur_time = cur_time.strftime("%H:%M:%S")
            print('▶ Episode #', n_epi, end=' → ')
            print('Score =', score, end='...')
            print('경로 길이:', len(env.actions), end=', ')
            if goal_ob_reward == 'finish':
                print('성공 여부: ', goal_ob_reward)
            else:
                print('성공 여부 : 실패', goal_ob_reward)
        
        score = 0.0
        print('→ pred_path:', env.actions)

    # 모든 에피소드 종료 후 결과 디스플레이
    # 코드 추가할 것!!!

In [10]:
# test 모드 실행
train_mode = False
test()

model path = /10000_model.pt
data/factory_order_test.csv used
→ pred_path: [(9, 4)]
▶ Episode # 1 → Score = -10.0...경로 길이: 1, 성공 여부 : 실패 True
→ pred_path: [(9, 4)]
▶ Episode # 2 → Score = -10.0...경로 길이: 1, 성공 여부 : 실패 True
→ pred_path: [(9, 4)]
▶ Episode # 3 → Score = -10.0...경로 길이: 1, 성공 여부 : 실패 True
→ pred_path: [(9, 4)]
▶ Episode # 4 → Score = -10.0...경로 길이: 1, 성공 여부 : 실패 True
→ pred_path: [(9, 4)]
▶ Episode # 5 → Score = -10.0...경로 길이: 1, 성공 여부 : 실패 True
→ pred_path: [(9, 4)]
▶ Episode # 6 → Score = -10.0...경로 길이: 1, 성공 여부 : 실패 True
→ pred_path: [(9, 4)]
▶ Episode # 7 → Score = -10.0...경로 길이: 1, 성공 여부 : 실패 True
→ pred_path: [(9, 4)]
▶ Episode # 8 → Score = -10.0...경로 길이: 1, 성공 여부 : 실패 True
→ pred_path: [(9, 4)]
▶ Episode # 9 → Score = -10.0...경로 길이: 1, 성공 여부 : 실패 True
→ pred_path: [(9, 4)]
▶ Episode # 10 → Score = -10.0...경로 길이: 1, 성공 여부 : 실패 True
→ pred_path: [(9, 4)]
▶ Episode # 11 → Score = -10.0...경로 길이: 1, 성공 여부 : 실패 True
→ pred_path: [(9, 4)]
▶ Episode # 12 → Score = -10.0...경

KeyboardInterrupt: ignored